## I tried turning `stop_times` to actual dates but it seems like seconds is easier to manipulate.
* 10_transit_bunching.ipynb contains timestamps attempts
* cd data-analyses/rt_segment_speeds && pip install -r requirements.txt && cd ../_shared_utils && make setup_env && cd ../gtfs_digest
* [Issue](https://github.com/cal-itp/data-analyses/issues/1099)

In [1]:
import geopandas as gpd
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers
from shared_utils import catalog_utils, rt_dates, rt_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS

# https://github.com/cal-itp/data-analyses/blob/main/_shared_utils/shared_utils/gtfs_analytics_data.yml
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)

In [2]:
import merge_data

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
may_date = "2024-05-22"

In [5]:
drop_for_preview = [
    "schedule_gtfs_dataset_key",
    "trip_instance_key",
    "shape_array_key",
    "feed_key",
    "trip_id",
]

### Get high frequency routes
* Group by mean frequency minutes for the operator-route-direction grain.

In [6]:
subset = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "direction_id",
    "route_primary_direction",
    "service_date",
    "frequency",
]

In [7]:
route_dir = merge_data.concatenate_schedule_by_route_direction([may_date])[subset]

In [8]:
route_dir["frequency_in_minutes"] = 60 / route_dir.frequency

In [9]:
route_freq_groupby = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "direction_id",
    "route_primary_direction",
]

In [10]:
high_frequency_routes = (
    route_dir.groupby(route_freq_groupby)
    .agg({"frequency_in_minutes": "mean"})
    .reset_index()
)

#### Grab routes in the 5th percentile of frequency for now.

In [11]:
high_frequency_routes["frequency_in_minutes"].describe(
    percentiles=[0.05, 0.1, 0.9, 0.95]
)

count   3417.00
mean     234.64
std      312.42
min        4.00
5%        17.65
10%       23.40
50%       97.71
90%      750.00
95%     1000.00
max     1250.00
Name: frequency_in_minutes, dtype: float64

In [12]:
high_frequency_routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3417 entries, 0 to 3416
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   schedule_gtfs_dataset_key  3417 non-null   object 
 1   route_id                   3417 non-null   object 
 2   direction_id               3417 non-null   float64
 3   route_primary_direction    3417 non-null   object 
 4   frequency_in_minutes       3417 non-null   float64
dtypes: float64(2), object(3)
memory usage: 133.6+ KB


In [13]:
high_frequency_routes2 = high_frequency_routes.loc[
    high_frequency_routes.frequency_in_minutes <= 17.65
]

In [14]:
high_frequency_routes2.route_id.nunique()

93

### Get trips of high frequency routes

In [15]:
TABLE = GTFS_DATA_DICT.schedule_downloads.trips

In [16]:
FILE = f"{COMPILED_CACHED_VIEWS}{TABLE}_{may_date}.parquet"

In [17]:
trips_subset = [
    "gtfs_dataset_key",
    "route_id",
    "trip_instance_key",
    "shape_array_key",
    "feed_key",
    "route_long_name",
    "direction_id",
]

In [18]:
trips = pd.read_parquet(FILE)[trips_subset].rename(
    columns={"gtfs_dataset_key": "schedule_gtfs_dataset_key"}
)

In [19]:
trips_freq_routes = pd.merge(
    trips,
    high_frequency_routes2,
    on=["schedule_gtfs_dataset_key", "route_id", "direction_id"],
    how="inner",
)

In [20]:
trips_freq_routes.head(2)

,schedule_gtfs_dataset_key,route_id,trip_instance_key,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes
0,cc53a0dbf5df90e3009b9cb5d89d80ba,4869,cd1d4fc457d3a3fff6e77e47336bbc98,7fca7ce64e1b773776b91ec1cf82c9ea,2cfdf0e33e9229d6b0ad124d956f5856,DASH Pico Union/Echo Park,0.00,Northbound,16.67
1,cc53a0dbf5df90e3009b9cb5d89d80ba,4869,180a069ab3aefcf8f3317a788b32c288,7fca7ce64e1b773776b91ec1cf82c9ea,2cfdf0e33e9229d6b0ad124d956f5856,DASH Pico Union/Echo Park,0.00,Northbound,16.67


In [21]:
trips_freq_routes.trip_instance_key.nunique()

20090

### `rt_stop_times2`: Get Stop Times of High Frequency Routes/Trips
* What's the difference btwn `trip_id` and `trip_instance_key`?

In [22]:
rt_stop_times = pd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/rt_vs_schedule/schedule_rt_stop_times_2024-05-22.parquet"
)

In [23]:
rt_stop_times.shape

(2601262, 7)

In [24]:
# Find only stop times of trips that belong to high frequency trips
rt_stop_times2 = pd.merge(
    rt_stop_times,
    trips_freq_routes,
    on=[
        "schedule_gtfs_dataset_key",
        "trip_instance_key",
    ],
    how="inner",
)

In [25]:
len(rt_stop_times) - len(rt_stop_times2)

1903905

In [26]:
rt_stop_times2.shape

(697357, 14)

In [27]:
rt_stop_times2.head(2)

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes
0,922552,258,2,61249.00,efbbd5293be71f7a5de0cf82b59febe1,50617e0d3c1bbedd9803836728767a69,61995,3730,e10d20177f6b29f7d2de52645301f18f,0e75eaae4dc791180f05782fa8825254,Main St & Santa Monica Blvd/UCLA,1.00,Southbound,16.08
1,922552,310,9,62012.00,efbbd5293be71f7a5de0cf82b59febe1,50617e0d3c1bbedd9803836728767a69,62826,3730,e10d20177f6b29f7d2de52645301f18f,0e75eaae4dc791180f05782fa8825254,Main St & Santa Monica Blvd/UCLA,1.00,Southbound,16.08


In [28]:
rt_stop_times2.trip_id.nunique(), rt_stop_times2.trip_instance_key.nunique()

(17213, 17213)

###  `rt_stop_times3`: Some scheduled arrival seconds span longer than a day: filter them out
* There are 86,400 seconds in a day

In [29]:
rt_stop_times2.scheduled_arrival_sec.describe()

count   697357.00
mean     50526.22
std      19329.72
min       9420.00
25%      34320.00
50%      49740.00
75%      64380.00
max     108431.00
Name: scheduled_arrival_sec, dtype: float64

In [30]:
len(rt_stop_times2.loc[rt_stop_times2.scheduled_arrival_sec > 86400])

27472

In [31]:
rt_stop_times3 = rt_stop_times2.loc[
    rt_stop_times2.scheduled_arrival_sec < 86400
].reset_index(drop=True)

In [32]:
len(rt_stop_times3)

669746

In [33]:
rt_stop_times3.scheduled_arrival_sec.describe()

count   669746.00
mean     48760.53
std      17580.67
min       9420.00
25%      33720.00
50%      48540.00
75%      62640.00
max      86399.00
Name: scheduled_arrival_sec, dtype: float64

In [34]:
rt_stop_times3.rt_arrival_sec.describe()

count   669746.00
mean     48828.92
std      17647.56
min          0.00
25%      33814.00
50%      48581.00
75%      62840.00
max      86399.00
Name: rt_arrival_sec, dtype: float64

### `rt_stop_times4`: Sort so stop sequence for the `operator-stop_id-route-id_direction_id` will be in order.
Help, which columns should I use to sort? Should I keep `feed_key` and `shape_array_key`?

In [35]:
# Rearrange: I want the stop sequence to be 1,2,3,4.
# stop ids can differ between trips of the same route and the same stop sequence is the same
rt_stop_times4 = rt_stop_times3.sort_values(
    by=[
        "schedule_gtfs_dataset_key",
        "route_id",
        "direction_id",
        "feed_key",
        "shape_array_key",
        "stop_sequence",
        "scheduled_arrival_sec",
    ]
).reset_index(drop=True)

### Calculate the difference btwn actual vs scheduled arrival.

In [36]:
def check_delay(df):
    df = df.assign(delay=df.rt_arrival_sec - df.scheduled_arrival_sec)

    print(df.delay.describe(percentiles=[0.05, 0.1, 0.9, 0.95]))

    max_delay_min = df.delay.max() / 60
    p95_delay_min = df.delay.quantile(q=0.95) / 60

    min_delay_min = df.delay.min() / 60
    p5_delay_min = df.delay.quantile(q=0.05) / 60

    print(f"min / max delay (minutes): {min_delay_min}, {max_delay_min}")
    print(f"5th / 95th delay (minutes): {p5_delay_min}, {p95_delay_min}")

    return df

In [37]:
rt_stop_times4 = check_delay(rt_stop_times4)

count   669746.00
mean        68.40
std       2841.12
min     -86381.00
5%        -167.00
10%       -107.00
50%         89.00
90%        515.00
95%        719.00
max      57878.00
Name: delay, dtype: float64
min / max delay (minutes): -1439.6833333333334, 964.6333333333333
5th / 95th delay (minutes): -2.783333333333333, 11.983333333333333


In [38]:
1439/60

23.983333333333334

#### `rt_stop_times5`: Filter out values in `delay` that ~1 hour zone

In [39]:
# Filter to only delays that are an hour or less
rt_stop_times5 = rt_stop_times4[rt_stop_times4["delay"] <= 60 * 60].reset_index(
    drop=True
)

In [40]:
# Filter to only delays that are no less than
rt_stop_times5 = rt_stop_times5[rt_stop_times5["delay"] >= -3600].reset_index(drop=True)

In [41]:
len(rt_stop_times4) - len(rt_stop_times5)

1100

In [42]:
len(rt_stop_times) - len(rt_stop_times5)

1932616

In [43]:
len(rt_stop_times2) - len(rt_stop_times5)

28711

In [44]:
rt_stop_times5.delay.describe()

count   668646.00
mean       158.11
std        306.54
min      -3559.00
25%        -24.00
50%         89.00
75%        269.00
max       3592.00
Name: delay, dtype: float64

### Calculate the actual headway the `operator-route-direction_id-stop_sequence-stop_id-` grain
* Do I need to include feed key and shape array key?

In [45]:
groupby_cols = [
    "schedule_gtfs_dataset_key",
    "feed_key",
    "shape_array_key",
    "route_id",
    "direction_id",
    "route_primary_direction",
    "stop_sequence",
    "stop_id",
]

In [46]:
rt_stop_times5["actual_headway"] = rt_stop_times5.groupby(groupby_cols)[
    "rt_arrival_sec"
].diff()

### Calculate scheduled headway

In [47]:
rt_stop_times5["schd_headway"] = rt_stop_times5.groupby(groupby_cols)[
    "scheduled_arrival_sec"
].diff()

In [48]:
rt_stop_times5.loc[rt_stop_times5.actual_headway.isna()].head(10).drop(columns=drop_for_preview)

,stop_id,stop_sequence,scheduled_arrival_sec,rt_arrival_sec,route_id,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay,actual_headway,schd_headway
0,4201,2,56100.00,56133,105-13172,Metro Local Line,0.00,Westbound,14.59,33.00,NaN,NaN
9,4198,3,56160.00,56211,105-13172,Metro Local Line,0.00,Westbound,14.59,51.00,NaN,NaN
19,10256,4,56280.00,56262,105-13172,Metro Local Line,0.00,Westbound,14.59,-18.00,NaN,NaN
29,10249,5,56340.00,56337,105-13172,Metro Local Line,0.00,Westbound,14.59,-3.00,NaN,NaN
39,36572,6,56460.00,56478,105-13172,Metro Local Line,0.00,Westbound,14.59,18.00,NaN,NaN
49,10243,7,56580.00,56589,105-13172,Metro Local Line,0.00,Westbound,14.59,9.00,NaN,NaN
59,10244,8,56700.00,56695,105-13172,Metro Local Line,0.00,Westbound,14.59,-5.00,NaN,NaN
69,10251,9,56820.00,56743,105-13172,Metro Local Line,0.00,Westbound,14.59,-77.00,NaN,NaN
79,10247,10,56940.00,56832,105-13172,Metro Local Line,0.00,Westbound,14.59,-108.00,NaN,NaN
89,10250,11,57060.00,56955,105-13172,Metro Local Line,0.00,Westbound,14.59,-105.00,NaN,NaN


### Delete out rows that are `nan`??
* I am not sure if `nans` impact calculations of the mean scheduled headway and whatnot?
* These `nans` are becuase the first `operator-route-stop_id-stop_sequence` combo won't have anything to compare it to.

In [49]:
rt_stop_times5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668646 entries, 0 to 668645
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   trip_id                    668646 non-null  object 
 1   stop_id                    668646 non-null  object 
 2   stop_sequence              668646 non-null  int64  
 3   scheduled_arrival_sec      668646 non-null  float64
 4   schedule_gtfs_dataset_key  668646 non-null  object 
 5   trip_instance_key          668646 non-null  object 
 6   rt_arrival_sec             668646 non-null  int64  
 7   route_id                   668646 non-null  object 
 8   shape_array_key            668646 non-null  object 
 9   feed_key                   668646 non-null  object 
 10  route_long_name            668646 non-null  object 
 11  direction_id               668646 non-null  float64
 12  route_primary_direction    668646 non-null  object 
 13  frequency_in_minutes       66

### `rt_stop_times6`: Delete out the rows in which `actual_headway` and `schd_headway` are `nan`: this is basically teh first row of each grain

In [50]:
rt_stop_times6 = rt_stop_times5.loc[~rt_stop_times5.actual_headway.isna()]

In [51]:
rt_stop_times6 = rt_stop_times6.loc[~rt_stop_times5.schd_headway.isna()].reset_index(
    drop=True
)

In [52]:
len(rt_stop_times5)-len(rt_stop_times6)

21439

### Find the mean scheduled headway for the `operator-route-direction_id-stop_sequence-stop_id-` grain

In [53]:
agg1 = (
    rt_stop_times6.groupby(groupby_cols)
    .agg({"schd_headway": "mean"})
    .reset_index()
    .rename(columns={"schd_headway": "avg_schd_headway_sec"})
)

In [54]:
agg1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19260 entries, 0 to 19259
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   schedule_gtfs_dataset_key  19260 non-null  object 
 1   feed_key                   19260 non-null  object 
 2   shape_array_key            19260 non-null  object 
 3   route_id                   19260 non-null  object 
 4   direction_id               19260 non-null  float64
 5   route_primary_direction    19260 non-null  object 
 6   stop_sequence              19260 non-null  int64  
 7   stop_id                    19260 non-null  object 
 8   avg_schd_headway_sec       19260 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 1.3+ MB


#### Merge

In [55]:
m1 = pd.merge(
    rt_stop_times6,
    agg1,
    on=groupby_cols,
)

In [56]:
len(rt_stop_times6) - len(m1)

0

### Find standard deviation: how far the actual headway is from the mean scheduled headway for the same grain above.


In [57]:
# Takes 1 minute
std_dev = (
    m1.groupby(groupby_cols)
    .apply(lambda x: (x["actual_headway"] - x["avg_schd_headway_sec"]).std())
    .reset_index(name="std_dev_headway")
)

#### Why are so many missing? Case 1
Why are some Groupby and transform: When using groupby and transform, if a group has only one element, the standard deviation is NaN (Not a Number). This is because standard deviation requires at least two data points to calculate.

In [58]:
std_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19260 entries, 0 to 19259
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   schedule_gtfs_dataset_key  19260 non-null  object 
 1   feed_key                   19260 non-null  object 
 2   shape_array_key            19260 non-null  object 
 3   route_id                   19260 non-null  object 
 4   direction_id               19260 non-null  float64
 5   route_primary_direction    19260 non-null  object 
 6   stop_sequence              19260 non-null  int64  
 7   stop_id                    19260 non-null  object 
 8   std_dev_headway            17781 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 1.3+ MB


In [59]:
std_dev.loc[std_dev.std_dev_headway.isna()].sample(1)

,schedule_gtfs_dataset_key,feed_key,shape_array_key,route_id,direction_id,route_primary_direction,stop_sequence,stop_id,std_dev_headway
17361,7cc0cb1871dfd558f11a2885c145d144,7f69c2fdaa134642f14064a0b64d1495,e28e3de81657b399d7e71e23eb625240,49,1.00,Northbound,7,15801,NaN


In [60]:
std_dev2 = std_dev.loc[~std_dev.std_dev_headway.isna()].reset_index(drop = True)

#### Investigate missing rows
* This one seems to have some very unrealistic time stamps, like the time between scheduled versus actual arrival times are more than an hour.

In [61]:
m1.loc[(m1.schedule_gtfs_dataset_key == "0666caf3ec1ecc96b74f4477ee4bc939") &
                  (m1.shape_array_key == "0688a14c97a2ebfe90f5674c1262d741") & 
                  (m1.route_id== "217-13172") &
                  (m1.direction_id == 1) &
                   (m1.stop_id == "15434")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay,actual_headway,schd_headway,avg_schd_headway_sec
161685,10217003302323-DEC23,15434,3,84360.00,0666caf3ec1ecc96b74f4477ee4bc939,bc7a2481002d5e5b5938a991db4e69d1,84408,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92,48.00,1818.00,1800.00,1800.00


#### `scheduled_arrival_sec` is 86,220 but `rt_arrival_sec` is 303.

In [62]:
rt_stop_times2.loc[(rt_stop_times2.schedule_gtfs_dataset_key == "0666caf3ec1ecc96b74f4477ee4bc939") &
                  (rt_stop_times2.shape_array_key == "0688a14c97a2ebfe90f5674c1262d741") & 
                  (rt_stop_times2.route_id== "217-13172") &
                  (rt_stop_times2.direction_id == 1) &
                   (rt_stop_times2.stop_id == "15434")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes
413908,10217003302253-DEC23,15434,3,82560.00,0666caf3ec1ecc96b74f4477ee4bc939,27d29b3a92104fdcb72b4095ef46fed6,82590,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92
419236,10217003302323-DEC23,15434,3,84360.00,0666caf3ec1ecc96b74f4477ee4bc939,bc7a2481002d5e5b5938a991db4e69d1,84408,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92
425286,10217003302432-DEC23,15434,3,88500.00,0666caf3ec1ecc96b74f4477ee4bc939,6699f5297ef2d670988b29937f33b56e,2031,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92
479447,10217003302354-DEC23,15434,3,86220.00,0666caf3ec1ecc96b74f4477ee4bc939,2f10227a381957bbf2b4f388e7f2a3e9,303,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92


In [63]:
rt_stop_times3.loc[(rt_stop_times3.schedule_gtfs_dataset_key == "0666caf3ec1ecc96b74f4477ee4bc939") &
                  (rt_stop_times3.shape_array_key == "0688a14c97a2ebfe90f5674c1262d741") & 
                  (rt_stop_times3.route_id== "217-13172") &
                  (rt_stop_times3.direction_id == 1) &
                   (rt_stop_times3.stop_id == "15434")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes
413484,10217003302253-DEC23,15434,3,82560.00,0666caf3ec1ecc96b74f4477ee4bc939,27d29b3a92104fdcb72b4095ef46fed6,82590,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92
417843,10217003302323-DEC23,15434,3,84360.00,0666caf3ec1ecc96b74f4477ee4bc939,bc7a2481002d5e5b5938a991db4e69d1,84408,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92
462159,10217003302354-DEC23,15434,3,86220.00,0666caf3ec1ecc96b74f4477ee4bc939,2f10227a381957bbf2b4f388e7f2a3e9,303,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92


In [64]:
rt_stop_times4.loc[(rt_stop_times4.schedule_gtfs_dataset_key == "0666caf3ec1ecc96b74f4477ee4bc939") &
                  (rt_stop_times4.shape_array_key == "0688a14c97a2ebfe90f5674c1262d741") & 
                  (rt_stop_times4.route_id== "217-13172") &
                  (rt_stop_times4.direction_id == 1) &
                   (rt_stop_times4.stop_id == "15434")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay
168196,10217003302253-DEC23,15434,3,82560.00,0666caf3ec1ecc96b74f4477ee4bc939,27d29b3a92104fdcb72b4095ef46fed6,82590,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92,30.00
168197,10217003302323-DEC23,15434,3,84360.00,0666caf3ec1ecc96b74f4477ee4bc939,bc7a2481002d5e5b5938a991db4e69d1,84408,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92,48.00
168198,10217003302354-DEC23,15434,3,86220.00,0666caf3ec1ecc96b74f4477ee4bc939,2f10227a381957bbf2b4f388e7f2a3e9,303,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92,-85917.00


In [65]:
rt_stop_times5.loc[(rt_stop_times5.schedule_gtfs_dataset_key == "0666caf3ec1ecc96b74f4477ee4bc939") &
                  (rt_stop_times5.shape_array_key == "0688a14c97a2ebfe90f5674c1262d741") & 
                  (rt_stop_times5.route_id== "217-13172") &
                  (rt_stop_times5.direction_id == 1) &
                   (rt_stop_times5.stop_id == "15434")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay,actual_headway,schd_headway
168062,10217003302253-DEC23,15434,3,82560.00,0666caf3ec1ecc96b74f4477ee4bc939,27d29b3a92104fdcb72b4095ef46fed6,82590,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92,30.00,NaN,NaN
168063,10217003302323-DEC23,15434,3,84360.00,0666caf3ec1ecc96b74f4477ee4bc939,bc7a2481002d5e5b5938a991db4e69d1,84408,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92,48.00,1818.00,1800.00


In [66]:
rt_stop_times6.loc[(rt_stop_times6.schedule_gtfs_dataset_key == "0666caf3ec1ecc96b74f4477ee4bc939") &
                  (rt_stop_times6.shape_array_key == "0688a14c97a2ebfe90f5674c1262d741") & 
                  (rt_stop_times6.route_id== "217-13172") &
                  (rt_stop_times6.direction_id == 1) &
                   (rt_stop_times6.stop_id == "15434")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay,actual_headway,schd_headway
161685,10217003302323-DEC23,15434,3,84360.00,0666caf3ec1ecc96b74f4477ee4bc939,bc7a2481002d5e5b5938a991db4e69d1,84408,217-13172,0688a14c97a2ebfe90f5674c1262d741,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,13.92,48.00,1818.00,1800.00


#### Missing rows Case 2
* There are only 2 rows, I guess there needs to be at least 3 rows to calculate the standard deviation since the first row of a combo won't have anything.

In [67]:
m1.loc[(m1.schedule_gtfs_dataset_key == "0666caf3ec1ecc96b74f4477ee4bc939") &
                  (m1.shape_array_key == "6f33c9cd019664d5085f94294aeacfd3") & 
                  (m1.route_id== "234-13172") &
                  (m1.direction_id == 1) &
                   (m1.stop_id == "15383")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay,actual_headway,schd_headway,avg_schd_headway_sec
179405,10234000780433-DEC23,15383,58,19380.00,0666caf3ec1ecc96b74f4477ee4bc939,3741cc1a8fd3d2ea5ddc59ba1766c0f5,19418,234-13172,6f33c9cd019664d5085f94294aeacfd3,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,14.42,38.00,1841.00,1800.00,1800.00


In [68]:
rt_stop_times2.loc[(rt_stop_times2.schedule_gtfs_dataset_key == "0666caf3ec1ecc96b74f4477ee4bc939") &
                  (rt_stop_times2.shape_array_key == "6f33c9cd019664d5085f94294aeacfd3") & 
                  (rt_stop_times2.route_id== "234-13172") &
                  (rt_stop_times2.direction_id == 1) &
                   (rt_stop_times2.stop_id == "15383")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes
256950,10234000780433-DEC23,15383,58,19380.00,0666caf3ec1ecc96b74f4477ee4bc939,3741cc1a8fd3d2ea5ddc59ba1766c0f5,19418,234-13172,6f33c9cd019664d5085f94294aeacfd3,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,14.42
482097,10234000780403-DEC23,15383,58,17580.00,0666caf3ec1ecc96b74f4477ee4bc939,bd2c7c473164147ae73920b7ea99c3b6,17577,234-13172,6f33c9cd019664d5085f94294aeacfd3,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,14.42


In [69]:
rt_stop_times3.loc[(rt_stop_times3.schedule_gtfs_dataset_key == "0666caf3ec1ecc96b74f4477ee4bc939") &
                  (rt_stop_times3.shape_array_key == "6f33c9cd019664d5085f94294aeacfd3") & 
                  (rt_stop_times3.route_id== "234-13172") &
                  (rt_stop_times3.direction_id == 1) &
                   (rt_stop_times3.stop_id == "15383")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes
256533,10234000780433-DEC23,15383,58,19380.00,0666caf3ec1ecc96b74f4477ee4bc939,3741cc1a8fd3d2ea5ddc59ba1766c0f5,19418,234-13172,6f33c9cd019664d5085f94294aeacfd3,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,14.42
462564,10234000780403-DEC23,15383,58,17580.00,0666caf3ec1ecc96b74f4477ee4bc939,bd2c7c473164147ae73920b7ea99c3b6,17577,234-13172,6f33c9cd019664d5085f94294aeacfd3,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,14.42


In [70]:
rt_stop_times4.loc[(rt_stop_times4.schedule_gtfs_dataset_key == "0666caf3ec1ecc96b74f4477ee4bc939") &
                  (rt_stop_times4.shape_array_key == "6f33c9cd019664d5085f94294aeacfd3") & 
                  (rt_stop_times4.route_id== "234-13172") &
                  (rt_stop_times4.direction_id == 1) &
                   (rt_stop_times4.stop_id == "15383")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay
186848,10234000780403-DEC23,15383,58,17580.00,0666caf3ec1ecc96b74f4477ee4bc939,bd2c7c473164147ae73920b7ea99c3b6,17577,234-13172,6f33c9cd019664d5085f94294aeacfd3,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,14.42,-3.00
186849,10234000780433-DEC23,15383,58,19380.00,0666caf3ec1ecc96b74f4477ee4bc939,3741cc1a8fd3d2ea5ddc59ba1766c0f5,19418,234-13172,6f33c9cd019664d5085f94294aeacfd3,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,14.42,38.00


In [71]:
rt_stop_times5.loc[(rt_stop_times5.schedule_gtfs_dataset_key == "0666caf3ec1ecc96b74f4477ee4bc939") &
                  (rt_stop_times5.shape_array_key == "6f33c9cd019664d5085f94294aeacfd3") & 
                  (rt_stop_times5.route_id== "234-13172") &
                  (rt_stop_times5.direction_id == 1) &
                   (rt_stop_times5.stop_id == "15383")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay,actual_headway,schd_headway
186708,10234000780403-DEC23,15383,58,17580.00,0666caf3ec1ecc96b74f4477ee4bc939,bd2c7c473164147ae73920b7ea99c3b6,17577,234-13172,6f33c9cd019664d5085f94294aeacfd3,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,14.42,-3.00,NaN,NaN
186709,10234000780433-DEC23,15383,58,19380.00,0666caf3ec1ecc96b74f4477ee4bc939,3741cc1a8fd3d2ea5ddc59ba1766c0f5,19418,234-13172,6f33c9cd019664d5085f94294aeacfd3,608992664173210532aa3e6cc573be2f,Metro Local Line,1.00,Southbound,14.42,38.00,1841.00,1800.00


#### Merge

In [72]:
m2 = pd.merge(
    m1,
    std_dev2,
    on=groupby_cols,
)

### Bunching coefficient is for the entire grain, rather than each row

In [73]:
m2["bunching_coefficient"] = m2.std_dev_headway / m2.avg_schd_headway_sec

In [74]:
m2.bunching_coefficient.describe()

count   645728.00
mean         0.72
std          0.43
min          0.00
25%          0.47
50%          0.63
75%          0.79
max          5.28
Name: bunching_coefficient, dtype: float64

In [75]:
m2["avg_schd_headway_min"] = (
    m2.avg_schd_headway_sec / 60
)

In [76]:
m2["actual_headway_min"] = m2.actual_headway / 60

In [77]:
m2["sched_headway_min"] = m2.schd_headway / 60

### Retain only one row f or the grain

In [78]:
bunching_by_stops = m2.drop_duplicates(subset=groupby_cols).reset_index(drop=True).drop(columns = ["actual_headway", "actual_headway_min", "sched_headway_min"])

In [79]:
len(m2) - len(bunching_by_stops)

627947

In [80]:
len(bunching_by_stops)

17781

In [81]:
bunching_by_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17781 entries, 0 to 17780
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   trip_id                    17781 non-null  object 
 1   stop_id                    17781 non-null  object 
 2   stop_sequence              17781 non-null  int64  
 3   scheduled_arrival_sec      17781 non-null  float64
 4   schedule_gtfs_dataset_key  17781 non-null  object 
 5   trip_instance_key          17781 non-null  object 
 6   rt_arrival_sec             17781 non-null  int64  
 7   route_id                   17781 non-null  object 
 8   shape_array_key            17781 non-null  object 
 9   feed_key                   17781 non-null  object 
 10  route_long_name            17781 non-null  object 
 11  direction_id               17781 non-null  float64
 12  route_primary_direction    17781 non-null  object 
 13  frequency_in_minutes       17781 non-null  flo

### Bunching Coefficient

In [82]:
def coefficient_frequency(row):
    if row.bunching_coefficient <= 0.21:
        return "Service provided like clockwork"
    elif 0.21 < row.bunching_coefficient <= 0.3:
        return "Vehicles slightly off headway"
    elif 0.3 < row.bunching_coefficient <= 0.39:
        return "Vehicles often off headway"
    elif 0.39 < row.bunching_coefficient <= 0.52:
        return "Irregular headways, with some bunching"
    elif 0.52 < row.bunching_coefficient <= 0.74:
        return "Frequent bunching"
    else:
        return "Most vehicles bunched"


bunching_by_stops["passenger_op_perspective"] = bunching_by_stops.apply(
    lambda x: coefficient_frequency(x), axis=1
)

In [109]:
bunching_by_stops.passenger_op_perspective.value_counts()/len(bunching_by_stops)

Most vehicles bunched                    0.35
Frequent bunching                        0.25
Irregular headways, with some bunching   0.18
Vehicles often off headway               0.09
Service provided like clockwork          0.09
Vehicles slightly off headway            0.04
Name: passenger_op_perspective, dtype: float64

### Missing Routes
* These routes and trips weren't found in the `stop_times` dataset.

In [84]:
high_freq_routes = set(high_frequency_routes2.route_id.unique().tolist())
remaining_routes = set(bunching_by_stops.route_id.unique().tolist())
high_freq_routes - remaining_routes

{'Beige-N',
 'Beige-S',
 'Blue Line',
 'Lot D',
 'Red Line',
 'West Field Garage',
 'eastvalley',
 'mposa'}

In [85]:
high_frequency_routes2.loc[high_frequency_routes2.route_id == 'West Field Garage'].head()

,schedule_gtfs_dataset_key,route_id,direction_id,route_primary_direction,frequency_in_minutes
597,2ff70dd1151d7532db40436f8228cd33,West Field Garage,0.00,Eastbound,10.00


#### Check out one route

In [86]:
rt_stop_times2.loc[rt_stop_times2.route_id == "West Field Garage"].head(1)

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes


In [87]:
rt_stop_times2.loc[rt_stop_times2.route_id == 'eastvalley'].head(1)

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes


In [88]:
trips_freq_routes.loc[trips_freq_routes.route_id == 'eastvalley'].head(1)

,schedule_gtfs_dataset_key,route_id,trip_instance_key,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes
19915,31152914d10e2d0977b8b2fabb167922,eastvalley,77a45a5bbd6e7b87ef670d90e16fdbc3,aebd18679bd2170ee61d7522bcfc11ab,7900b8b29688af30a699822f84ae2753,East Valley Shuttle,1.00,Eastbound,16.05


In [89]:
rt_stop_times2.loc[(rt_stop_times2.trip_instance_key == "77a45a5bbd6e7b87ef670d90e16fdbc3")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes


In [90]:
rt_stop_times.loc[(rt_stop_times.trip_instance_key == "77a45a5bbd6e7b87ef670d90e16fdbc3")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec


#### Second Route

In [91]:
trips_freq_routes.loc[trips_freq_routes.route_id =='Beige-S'].sample(2)

,schedule_gtfs_dataset_key,route_id,trip_instance_key,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes
2380,8a1405af8da1379acc062e346187ac98,Beige-S,92a2015d35b766955a64094643f88435,93312729ad2928439c75cbbd97191b66,3e22f1090d0d12096ee943c621298225,Coliseum to Oakland Airport,1.00,Southbound,7.90
2274,8a1405af8da1379acc062e346187ac98,Beige-S,e2ec930bd93a37260f6807108530a407,93312729ad2928439c75cbbd97191b66,3e22f1090d0d12096ee943c621298225,Coliseum to Oakland Airport,1.00,Southbound,7.90


In [92]:
rt_stop_times2.loc[(rt_stop_times2.trip_instance_key == "96e2844c6ce2bcc5ede547c32c220efb")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes


In [93]:
rt_stop_times2.loc[(rt_stop_times2.trip_instance_key == "4f9e575f015b6bf05e69af9da50d9579")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes


### Fillmore Test

In [94]:
more_values = ["scheduled_arrival_sec","std_dev_headway","avg_schd_headway_sec","schd_headway","rt_arrival_sec"]

In [95]:
drop_for_preview = drop_for_preview + more_values

In [96]:
fillmore = bunching_by_stops.loc[bunching_by_stops.route_long_name == "FILLMORE"]

In [97]:
fillmore.passenger_op_perspective.value_counts()

Most vehicles bunched                     120
Frequent bunching                          24
Irregular headways, with some bunching      1
Vehicles slightly off headway               1
Service provided like clockwork             1
Name: passenger_op_perspective, dtype: int64

In [98]:
fillmore.drop(columns = drop_for_preview)

,stop_id,stop_sequence,route_id,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay,bunching_coefficient,avg_schd_headway_min,passenger_op_perspective
13438,14630,2,22,FILLMORE,0.00,Southbound,7.61,161.00,0.49,29.18,"Irregular headways, with some bunching"
13439,14609,7,22,FILLMORE,0.00,Southbound,7.61,-53.00,0.73,22.60,Frequent bunching
13440,14627,8,22,FILLMORE,0.00,Southbound,7.61,-98.00,0.99,22.60,Most vehicles bunched
13441,16754,9,22,FILLMORE,0.00,Southbound,7.61,115.00,0.82,24.17,Most vehicles bunched
13442,16491,10,22,FILLMORE,0.00,Southbound,7.61,43.00,0.73,24.17,Frequent bunching
13443,16488,11,22,FILLMORE,0.00,Southbound,7.61,21.00,0.68,24.15,Frequent bunching
13444,16493,12,22,FILLMORE,0.00,Southbound,7.61,-20.00,0.67,24.12,Frequent bunching
13445,13086,13,22,FILLMORE,0.00,Southbound,7.61,-58.00,0.66,24.09,Frequent bunching
13446,14605,14,22,FILLMORE,0.00,Southbound,7.61,-113.00,0.74,24.05,Frequent bunching
13447,14624,15,22,FILLMORE,0.00,Southbound,7.61,-84.00,0.67,24.02,Frequent bunching


In [99]:
fillmore.tail(1)

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay,schd_headway,avg_schd_headway_sec,std_dev_headway,bunching_coefficient,avg_schd_headway_min,passenger_op_perspective
13584,11490047_M31,17769,7,67772.00,7cc0cb1871dfd558f11a2885c145d144,51672d1ed74ce3e66e4bfc273e6e9931,68087,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,315.00,25957.00,10023.50,11333.78,1.13,167.06,Most vehicles bunched


#### What do you do with scheduled headways that are pretty extreme compared to the other rows?
* One headway is scheduled for 7 hours?

In [101]:
436/60

7.266666666666667

In [102]:
m2.loc[(m2.schedule_gtfs_dataset_key == "7cc0cb1871dfd558f11a2885c145d144") &
                  (m2.shape_array_key == "fefbc78a6cf676d7fbd1d25b61ef7bfb") & 
                  (m2.route_id== "22") &
                  (m2.direction_id == 1) &
                   (m2.stop_id == "17769")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay,actual_headway,schd_headway,avg_schd_headway_sec,std_dev_headway,bunching_coefficient,avg_schd_headway_min,actual_headway_min,sched_headway_min
443917,11490047_M31,17769,7,67772.00,7cc0cb1871dfd558f11a2885c145d144,51672d1ed74ce3e66e4bfc273e6e9931,68087,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,315.00,26178.00,25957.00,10023.50,11333.78,1.13,167.06,436.30,432.62
443918,11490090_M31,17769,7,72092.00,7cc0cb1871dfd558f11a2885c145d144,6aaf49dfccb82109a1f77471fee6e43a,71689,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,-403.00,3602.00,4320.00,10023.50,11333.78,1.13,167.06,60.03,72.00
443919,11490091_M31,17769,7,73892.00,7cc0cb1871dfd558f11a2885c145d144,e8ff0c3ae23fd4ba4ab3d251d0c11069,72690,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,-1202.00,1001.00,1800.00,10023.50,11333.78,1.13,167.06,16.68,30.00
443920,11490099_M31,17769,7,81909.00,7cc0cb1871dfd558f11a2885c145d144,2c824de8cf7a1b5338f76d2501e6ebd5,81349,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,-560.00,8659.00,8017.00,10023.50,11333.78,1.13,167.06,144.32,133.62


In [103]:
rt_stop_times2.loc[(rt_stop_times2.schedule_gtfs_dataset_key == "7cc0cb1871dfd558f11a2885c145d144") &
                  (rt_stop_times2.shape_array_key == "fefbc78a6cf676d7fbd1d25b61ef7bfb") & 
                  (rt_stop_times2.route_id== "22") &
                  (rt_stop_times2.direction_id == 1) &
                   (rt_stop_times2.stop_id == "17769")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes
528875,11490047_M31,17769,7,67772.00,7cc0cb1871dfd558f11a2885c145d144,51672d1ed74ce3e66e4bfc273e6e9931,68087,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60
529753,11490091_M31,17769,7,73892.00,7cc0cb1871dfd558f11a2885c145d144,e8ff0c3ae23fd4ba4ab3d251d0c11069,72690,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60
556189,11490099_M31,17769,7,81909.00,7cc0cb1871dfd558f11a2885c145d144,2c824de8cf7a1b5338f76d2501e6ebd5,81349,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60
688955,11490090_M31,17769,7,72092.00,7cc0cb1871dfd558f11a2885c145d144,6aaf49dfccb82109a1f77471fee6e43a,71689,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60
689073,11490074_M31,17769,7,41815.00,7cc0cb1871dfd558f11a2885c145d144,59dbf32fbb998fc9837ad6fe10146b64,41909,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60


In [104]:
rt_stop_times3.loc[(rt_stop_times3.schedule_gtfs_dataset_key == "7cc0cb1871dfd558f11a2885c145d144") &
                  (rt_stop_times3.shape_array_key == "fefbc78a6cf676d7fbd1d25b61ef7bfb") & 
                  (rt_stop_times3.route_id== "22") &
                  (rt_stop_times3.direction_id == 1) &
                   (rt_stop_times3.stop_id == "17769")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes
508536,11490047_M31,17769,7,67772.00,7cc0cb1871dfd558f11a2885c145d144,51672d1ed74ce3e66e4bfc273e6e9931,68087,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60
509366,11490091_M31,17769,7,73892.00,7cc0cb1871dfd558f11a2885c145d144,e8ff0c3ae23fd4ba4ab3d251d0c11069,72690,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60
534538,11490099_M31,17769,7,81909.00,7cc0cb1871dfd558f11a2885c145d144,2c824de8cf7a1b5338f76d2501e6ebd5,81349,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60
661545,11490090_M31,17769,7,72092.00,7cc0cb1871dfd558f11a2885c145d144,6aaf49dfccb82109a1f77471fee6e43a,71689,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60
661663,11490074_M31,17769,7,41815.00,7cc0cb1871dfd558f11a2885c145d144,59dbf32fbb998fc9837ad6fe10146b64,41909,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60


In [105]:
rt_stop_times4.loc[(rt_stop_times4.schedule_gtfs_dataset_key == "7cc0cb1871dfd558f11a2885c145d144") &
                  (rt_stop_times4.shape_array_key == "fefbc78a6cf676d7fbd1d25b61ef7bfb") & 
                  (rt_stop_times4.route_id== "22") &
                  (rt_stop_times4.direction_id == 1) &
                   (rt_stop_times4.stop_id == "17769")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay
462155,11490074_M31,17769,7,41815.00,7cc0cb1871dfd558f11a2885c145d144,59dbf32fbb998fc9837ad6fe10146b64,41909,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,94.00
462156,11490047_M31,17769,7,67772.00,7cc0cb1871dfd558f11a2885c145d144,51672d1ed74ce3e66e4bfc273e6e9931,68087,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,315.00
462157,11490090_M31,17769,7,72092.00,7cc0cb1871dfd558f11a2885c145d144,6aaf49dfccb82109a1f77471fee6e43a,71689,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,-403.00
462158,11490091_M31,17769,7,73892.00,7cc0cb1871dfd558f11a2885c145d144,e8ff0c3ae23fd4ba4ab3d251d0c11069,72690,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,-1202.00
462159,11490099_M31,17769,7,81909.00,7cc0cb1871dfd558f11a2885c145d144,2c824de8cf7a1b5338f76d2501e6ebd5,81349,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,-560.00


In [106]:
41815/60/60

11.615277777777777

In [107]:
rt_stop_times5.loc[(rt_stop_times5.schedule_gtfs_dataset_key == "7cc0cb1871dfd558f11a2885c145d144") &
                  (rt_stop_times5.shape_array_key == "fefbc78a6cf676d7fbd1d25b61ef7bfb") & 
                  (rt_stop_times5.route_id== "22") &
                  (rt_stop_times5.direction_id == 1) &
                   (rt_stop_times5.stop_id == "17769")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay,actual_headway,schd_headway
461677,11490074_M31,17769,7,41815.00,7cc0cb1871dfd558f11a2885c145d144,59dbf32fbb998fc9837ad6fe10146b64,41909,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,94.00,NaN,NaN
461678,11490047_M31,17769,7,67772.00,7cc0cb1871dfd558f11a2885c145d144,51672d1ed74ce3e66e4bfc273e6e9931,68087,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,315.00,26178.00,25957.00
461679,11490090_M31,17769,7,72092.00,7cc0cb1871dfd558f11a2885c145d144,6aaf49dfccb82109a1f77471fee6e43a,71689,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,-403.00,3602.00,4320.00
461680,11490091_M31,17769,7,73892.00,7cc0cb1871dfd558f11a2885c145d144,e8ff0c3ae23fd4ba4ab3d251d0c11069,72690,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,-1202.00,1001.00,1800.00
461681,11490099_M31,17769,7,81909.00,7cc0cb1871dfd558f11a2885c145d144,2c824de8cf7a1b5338f76d2501e6ebd5,81349,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,-560.00,8659.00,8017.00


In [108]:
rt_stop_times5.loc[(rt_stop_times5.schedule_gtfs_dataset_key == "7cc0cb1871dfd558f11a2885c145d144") &
                  (rt_stop_times5.shape_array_key == "fefbc78a6cf676d7fbd1d25b61ef7bfb") & 
                  (rt_stop_times5.route_id== "22") &
                  (rt_stop_times5.direction_id == 1) &
                   (rt_stop_times5.stop_id == "17769")]

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec,route_id,shape_array_key,feed_key,route_long_name,direction_id,route_primary_direction,frequency_in_minutes,delay,actual_headway,schd_headway
461677,11490074_M31,17769,7,41815.00,7cc0cb1871dfd558f11a2885c145d144,59dbf32fbb998fc9837ad6fe10146b64,41909,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,94.00,NaN,NaN
461678,11490047_M31,17769,7,67772.00,7cc0cb1871dfd558f11a2885c145d144,51672d1ed74ce3e66e4bfc273e6e9931,68087,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,315.00,26178.00,25957.00
461679,11490090_M31,17769,7,72092.00,7cc0cb1871dfd558f11a2885c145d144,6aaf49dfccb82109a1f77471fee6e43a,71689,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,-403.00,3602.00,4320.00
461680,11490091_M31,17769,7,73892.00,7cc0cb1871dfd558f11a2885c145d144,e8ff0c3ae23fd4ba4ab3d251d0c11069,72690,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,-1202.00,1001.00,1800.00
461681,11490099_M31,17769,7,81909.00,7cc0cb1871dfd558f11a2885c145d144,2c824de8cf7a1b5338f76d2501e6ebd5,81349,22,fefbc78a6cf676d7fbd1d25b61ef7bfb,7f69c2fdaa134642f14064a0b64d1495,FILLMORE,1.00,Northbound,7.60,-560.00,8659.00,8017.00
